In [2]:
from imdb import IMDb
import pandas as pd

ia = IMDb()
filme = ia.search_movie('Matrix')[0]
ia.update(filme)
print(filme)


The Matrix


In [6]:
from imdb import IMDb

# Cria o objeto IMDb
ia = IMDb()

# Busca pelo título (exemplo: "The Matrix")
resultados = ia.search_movie("The Matrix")
filme = resultados[0]

# Atualiza com o pacote principal para garantir mais informações
ia.update(filme, info=['main'])

# Acessa Título, Ano e Gêneros
titulo = filme.get('title')
ano = filme.get('year')
generos = filme.get('genres')

print(f"Título: {titulo}")
print(f"Ano: {ano}")
print(f"Gêneros: {generos}")


Título: The Matrix
Ano: 1999
Gêneros: None


In [16]:
resultados = ia.search_movie("The Matrix")

# Pega o primeiro resultado
filme = resultados[0]

# Veja o que veio
print(filme)


The Matrix


In [41]:
ia.update(filme, info=['year'])


In [46]:
from imdb import IMDb

# Cria o objeto IMDb, forçando usar o 'web' parser
ia = IMDb(accessSystem='web')

resultados = ia.search_movie("The Matrix")
filme = resultados[0]

# Força carregar tudo
ia.update(filme, info=None)

print("\nKeys disponíveis:", filme.keys())
print("\nGêneros:", filme.get('genres'))
print("Diretores:", filme.get('directors'))
print("Ano:", filme.get('year'))



Keys disponíveis: ['title', 'kind', 'cover url', 'year', 'plot', 'synopsis', 'canonical title', 'long imdb title', 'long imdb canonical title', 'smart canonical title', 'smart long imdb canonical title', 'full-size cover url']

Gêneros: None
Diretores: None
Ano: 1999


In [112]:
import requests
from bs4 import BeautifulSoup

def buscar_detalhes_fallback(titulo):
    base_url = "https://www.imdb.com"
    headers = {"User-Agent": "Mozilla/5.0"}

    busca_url = f"{base_url}/find?q={titulo.replace(' ', '+')}&s=tt&ttype=ft"
    resposta = requests.get(busca_url, headers=headers)
    soup = BeautifulSoup(resposta.text, "html.parser")

    resultado = soup.select_one("a[href^='/title/tt']")
    print(resultado)
    if not resultado:
        print(f"Nenhum resultado encontrado para '{titulo}'.")
        return None

    link_filme = base_url + resultado['href']
    resposta_filme = requests.get(link_filme, headers=headers)
    soup_filme = BeautifulSoup(resposta_filme.text, "html.parser")

    titulo_tag = soup_filme.find("h1")
    titulo_final = titulo_tag.text.strip() if titulo_tag else "N/A"

    # ✅ Ano
    ano_tag = soup_filme.select_one("ul[data-testid='hero-title-block__metadata'] li span")
    ano = ano_tag.text.strip() if ano_tag else "N/A"

    # ✅ Gêneros
    generos = [s.text.strip() for s in soup_filme.select(
        "li[data-testid='storyline-genres'] a.ipc-metadata-list-item__list-content-item--link"
    )]
    if not generos:
        generos = [s.text.strip() for s in soup_filme.select(
            "div.ipc-chip-list__scroller a.ipc-chip span.ipc-chip__text"
        )]
    if not generos:
        generos = ["N/A"]

    # ✅ Direção
    diretores = [
        a.text.strip()
        for a in soup_filme.select(
            "li[data-testid='title-pc-principal-credit'] div.ipc-metadata-list-item__content-container ul li a.ipc-metadata-list-item__list-content-item--link"
        )
    ]
    if not diretores:
        diretores = ["N/A"]

    detalhes = {
        "Título": titulo_final,
        "Ano": ano,
        "Gêneros/Temas": generos,
        "Diretores": diretores,
        "Link IMDb": link_filme
    }

    return detalhes

# Teste
detalhes = buscar_detalhes_fallback("The Matrix")
if detalhes:
    for chave, valor in detalhes.items():
        print(f"{chave}: {valor}")
else:
    print("Nada encontrado.")


<a aria-disabled="false" class="ipc-metadata-list-summary-item__t" href="/title/tt0133093/?ref_=fn_ttl_ttl_1" tabindex="0">Matrix</a>
Título: Matrix
Ano: N/A
Gêneros/Temas: ['Action Epic', 'Artificial Intelligence', 'Cyberpunk', 'Dystopian Sci-Fi', 'Gun Fu', 'Martial Arts', 'Sci-Fi Epic', 'Action', 'Sci-Fi']
Diretores: ['Lana Wachowski', 'Lilly Wachowski', 'Lilly Wachowski', 'Lana Wachowski', 'Keanu Reeves', 'Laurence Fishburne', 'Carrie-Anne Moss', 'Lana Wachowski', 'Lilly Wachowski', 'Lilly Wachowski', 'Lana Wachowski', 'Keanu Reeves', 'Laurence Fishburne', 'Carrie-Anne Moss']
Link IMDb: https://www.imdb.com/title/tt0133093/?ref_=fn_ttl_ttl_1
